## 课堂笔记
### 基本的RNN原理和实现
全连接层的权重相对是最多的

在卷积网络中卷积核共享，可以减少权重

rnn  主要设计用来处理带有序列性质的数据  并且具有权重共享的概念（数据前后之间具有时间序列关系） ex：自然语言

rnn cell 本质是一个线形层   实现向量维度的改变 这个线性层会一直反复的参与运算（权重在不同的序列之间共享），处理不同的序列信息。

融合的概念：例如两者之间求和  做乘积 都可以叫做融合

rnn中的激活函数：使用tanh比较多

独热向量one-hot：缺点：1. 维度过高 2.向量稀疏 3. 硬编码

# 使用RNNcell单元 自己编写循环处理

In [37]:
# 导入需要的依赖包
import torch

In [38]:
# 进行属性值的配置
'''
序列长度seq_len表示的是一共有多少个数据样本  
输入维度input_size表示的一个样本里的向量维度是多少
隐藏维度hidden_size表示的一个样本经过rnn计算以后新的维度
'''
batch_size  = 1
seq_len = 3
input_size = 4
hidden_size = 2

In [39]:
# 定义rnn cell
cell = torch.nn.RNNCell(input_size= input_size,hidden_size= hidden_size)

In [40]:
# 生成随机的数据
dataset = torch.randn(seq_len, batch_size, input_size)
# 指的是h0
hidden = torch.zeros(batch_size,hidden_size)

In [41]:
# 自己编写循环进行rnn计算
# 根据代码可知使用的是tanh激活函数
# note：这里使用的是一个cell因此out和hidden都是hidden

for idx, input in enumerate(dataset):
    print('='*20, idx,'='*20)
    # 输入是一个1*4 的矩阵  batch_size, input_size 这表示的是一个序列
    print('Input size:', input.shape) 
    # 进行计算 hidden是上一个序列输入的hidden  
    hidden = cell(input,hidden)
    # 输出维度是一个1*2的矩阵  batch_size,hidden_size 
    print('output_size:', hidden.shape)
    print(hidden)
    

==================== 0 ====================
Input size: torch.Size([1, 4])
output_size: torch.Size([1, 2])
tensor([[-0.0512,  0.5675]], grad_fn=<TanhBackward>)
==================== 1 ====================
Input size: torch.Size([1, 4])
output_size: torch.Size([1, 2])
tensor([[-0.6816, -0.5774]], grad_fn=<TanhBackward>)
==================== 2 ====================
Input size: torch.Size([1, 4])
output_size: torch.Size([1, 2])
tensor([[0.8066, 0.4792]], grad_fn=<TanhBackward>)


# 使用RNN直接进行计算

In [42]:
# 直接使用rnn进行计算
# input 输入 整个序列 相比于使用cell的方式，不需要自己手写循环遍历序列
# output 输出两个值 分别为out和hidden  out表示每一个序列对应的值  hidden即为综合之前所有序列在每一层的最终值

### 通过代码观察输入、输出维度的变化

In [43]:
# 引入依赖包
import torch

In [44]:
# 设置参数值
batch_size = 1
seq_len = 3
input_size = 4
hidden_size = 2
num_layers = 1

In [45]:
# 随机创建数据
# input 即为输入的数据  
# hidden 表示h0
inputs = torch.randn(seq_len,batch_size,input_size)
hidden = torch.zeros(num_layers,batch_size,hidden_size)

In [46]:
# 创建模型 batch_first如果是true 输入中的第一个维度应该改为batch_size
rnn = torch.nn.RNN(input_size=input_size, hidden_size = hidden_size,num_layers = num_layers,batch_first = False)


In [47]:
# 执行计算
out,hidden = rnn(inputs, hidden)

In [48]:
print('output size: ',out.shape) # seq_len*batch_size*hidden_size
# 关于输出维度的一点说明  seq_len 表示的是样本个数 所以在rnn过程中不会法神改变
# hidden_size表示某个序列的向量维度 会随着rnn计算的进行发生变化
print('output:',out)
print('hidden size: ',hidden.shape)# num_layers*batch_size*hidden_size
print('hidden:',hidden)

output size:  torch.Size([3, 1, 2])
output: tensor([[[ 0.1882,  0.2237]],

        [[ 0.1690, -0.1969]],

        [[ 0.5927, -0.5091]]], grad_fn=<StackBackward>)
hidden size:  torch.Size([1, 1, 2])
hidden: tensor([[[ 0.5927, -0.5091]]], grad_fn=<StackBackward>)



### 利用rnn进行seq2seq的序列转换
hello->ohlol转化

#### 使用rnncell自己编写循环进行转化的实现

In [49]:
# 导入依赖包
import torch

In [50]:
# 设置配置的初始值
input_size = 4 # 只有四个不同的字符 因此使用one-hot编码方式四个维度即可
hidden_size = 4
batch_size = 1

In [51]:
# 创建样本数据

idx2char = ['e', 'h', 'l', 'o']
# x y 中的都表示在字典idx2char中的索引
x_data = [1, 0, 2, 2, 3] # hello
y_data = [3, 1, 2, 3, 2] # 0hlol

# 进行one-hot 编码
one_hot_lookup = [[1, 0, 0, 0],[0, 1, 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]]
x_one_hot = [one_hot_lookup[x] for x in x_data]
# 调整数据维度 5*1*4 改为 seq_len*batch_size * input_size
inputs = torch.Tensor(x_one_hot).view(-1, batch_size, input_size)
# labels 输出值的标签seq_len*1 
labels = torch.LongTensor(y_data).view(-1, 1)

In [52]:
# 定义模型1 rnncell

class Model_rnncell(torch.nn.Module):
    def __init__(self, input_size, hidden_size, batch_size):
      super(Model_rnncell,self).__init__()# 老师的课件中有误 修正一下
      self.input_size = input_size
      self.hidden_size = hidden_size
      self.batch_size = batch_size
      # 创建计算模型
      self.rnncell = torch.nn.RNNCell(input_size = self.input_size,hidden_size = self.hidden_size)
    
    # 前向传播
    def forward(self, input, hidden):
        # 这里的参数hidden指的是上一个序列处理得到的hidden
        hidden = self.rnncell(input,hidden)
        return hidden
    
    # 实现初始化一个h0
    def init_hidden(self):
        return torch.zeros(self.batch_size,self.hidden_size)





In [53]:
# 实例化模型
rnn_cell = Model_rnncell(input_size=input_size,hidden_size=hidden_size,batch_size=batch_size)

In [54]:
# 创建损失
criterion = torch.nn.CrossEntropyLoss()

In [55]:
# 创建优化器
optimizer = torch.optim.Adam(rnn_cell.parameters(), lr=0.1)

In [56]:
# 开始模型训练 通过rnn实现两个维度的映射
epochs = 20
for epoch in range(epochs):# 15个epoch
    # 初始化loss
    loss = 0
    # 优化器梯度归零
    optimizer.zero_grad()
    # 生成初始化隐藏维度 参数在初始化模型的时候已经传入
    hidden = rnn_cell.init_hidden()
    print('Predicted string: ', end = ' ')
    # 每次循环取inputs中的一个序列 循环结束 序列取完
    for input,label in zip(inputs,labels):
        # 使用模型进行计算
        hidden = rnn_cell(input,hidden)
        # 每一步都可以看作构建计算图的一部分
        loss += criterion(hidden, label)
        # 找到最大的数所在的索引 idx是一个tensor类型数据
        _,idx = hidden.max(dim =1)
        print(idx2char[idx.item()],end = ' ')
    # 反向传播计算梯度信息
    loss.backward()
    # 优化器进行优化
    optimizer.step()
    print(', Epoch [%d/%d] loss=%.4f' % (epoch+1, epochs,loss.item()))   
        
    

Predicted string:  l h e e l , Epoch [1/20] loss=7.4501
Predicted string:  l h e h l , Epoch [2/20] loss=5.9269
Predicted string:  l h l h l , Epoch [3/20] loss=4.8785
Predicted string:  l h l h l , Epoch [4/20] loss=4.1979
Predicted string:  o h l o l , Epoch [5/20] loss=3.7218
Predicted string:  o h l o l , Epoch [6/20] loss=3.3757
Predicted string:  o h l o l , Epoch [7/20] loss=3.0821
Predicted string:  o h l o l , Epoch [8/20] loss=2.8174
Predicted string:  o h l o l , Epoch [9/20] loss=2.5972
Predicted string:  o h l o l , Epoch [10/20] loss=2.4399
Predicted string:  o h l o l , Epoch [11/20] loss=2.3268
Predicted string:  o h l o l , Epoch [12/20] loss=2.2222
Predicted string:  o h l o l , Epoch [13/20] loss=2.1206
Predicted string:  o h l o l , Epoch [14/20] loss=2.0418
Predicted string:  o h l o l , Epoch [15/20] loss=1.9956
Predicted string:  o h l o l , Epoch [16/20] loss=1.9650
Predicted string:  o h l o l , Epoch [17/20] loss=1.9371
Predicted string:  o h l o l , Epoch [18

#### 使用rnn创建模型

In [57]:
# 导入依赖包
import torch

In [58]:
# 初始化配置参数
input_size = 4 # 只有四个不同的字符 因此使用one-hot编码方式四个维度即可
hidden_size = 4
batch_size = 1
seq_len = 5 # 直接使用rnn模块 所以需要给出每一层需要处理的序列长度（样本个数 这里是五个字母 所以设置为5）
num_layers = 1

In [59]:
# 初始化样本数据

idx2char = ['e', 'h', 'l', 'o']
# x y 中的都表示在字典idx2char中的索引
x_data = [1, 0, 2, 2, 3] # hello
y_data = [3, 1, 2, 3, 2] # 0hlol

# 进行one-hot 编码
one_hot_lookup = [[1, 0, 0, 0],[0, 1, 0, 0],[0, 0, 1, 0],[0, 0, 0, 1]]
x_one_hot = [one_hot_lookup[x] for x in x_data]
# 调整数据维度 5*1*4 改为 seq_len*batch_size * input_size
inputs = torch.Tensor(x_one_hot).view(seq_len, batch_size, input_size)
# labels 输出值的标签seq_len
labels = torch.LongTensor(y_data) 

In [60]:
# 创建基于rnn的模型

class RnnModel(torch.nn.Module):
    def __init__(self,input_size,hidden_size,batch_size,num_layers =1 ):# rnn默认一层
        super(RnnModel,self).__init__()
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.input_size = input_size
        # 创建rnn模型 需要给出的三个参数
        self.rnn = torch.nn.RNN(input_size = input_size,hidden_size = hidden_size,num_layers = num_layers)

    # 前向传播过程
    def forward(self,input):
        # 这里首先创建一个初始化的h0
        hidden = torch.zeros(self.num_layers,self.batch_size, self.hidden_size)
        # note：对结果取用的一点说明 out指的是每一个序列 
        # 再上一个模型中 进行训练得出的隐藏序列是每一个序列最后输出的结果
        out,_ = self.rnn(input,hidden)
        return out.view(-1,self.hidden_size)


In [61]:
# 模型实例化
rnnModel = RnnModel(input_size,hidden_size,batch_size,num_layers)

In [62]:
# 创建损失
criterion = torch.nn.CrossEntropyLoss()

In [63]:
# 创建优化器
optimizer = torch.optim.Adam(rnnModel.parameters(), lr=0.05)

In [64]:
# 训练过程
epochs = 15
for epoch in range (epochs):
    # 梯度归零
    optimizer.zero_grad()
    # 计算模型输出
    outputs = rnnModel(inputs)
    # 计算模型损失
    loss = criterion(outputs,labels)
    # 反向传播梯度
    loss.backward()
    # 使用优化器进行迭代
    optimizer.step()

    # 得到结果最大的索引
    _, idx = outputs.max(dim = 1)
    idx = idx.data.numpy()
    print('Predicted: ', ''.join([idx2char[x] for x in idx]), end='')
    print(', Epoch [%d/%d] loss = %.3f' % (epoch + 1, epochs,loss.item()))



Predicted:  hllll, Epoch [1/15] loss = 1.345
Predicted:  lllll, Epoch [2/15] loss = 1.228
Predicted:  lllll, Epoch [3/15] loss = 1.131
Predicted:  ohlll, Epoch [4/15] loss = 1.047
Predicted:  ohlll, Epoch [5/15] loss = 0.978
Predicted:  ohlll, Epoch [6/15] loss = 0.927
Predicted:  ohlol, Epoch [7/15] loss = 0.886
Predicted:  ohlol, Epoch [8/15] loss = 0.851
Predicted:  ohlol, Epoch [9/15] loss = 0.820
Predicted:  ohlol, Epoch [10/15] loss = 0.790
Predicted:  ohlol, Epoch [11/15] loss = 0.763
Predicted:  ohlll, Epoch [12/15] loss = 0.738
Predicted:  ohlll, Epoch [13/15] loss = 0.717
Predicted:  ohlll, Epoch [14/15] loss = 0.697
Predicted:  ohlll, Epoch [15/15] loss = 0.679


#### 使用词嵌入解决one-hot编码问题

In [65]:
# 引入依赖包
import torch

In [66]:
# 设置基本的参数值

num_class = 4 # 类别数量 
input_size = 4 # 输入的一个序列的维度（表示一个字母的维度  我们这里使用one-hot是四维的）
hidden_size = 8 # 隐藏层的维度
embedding_size = 10 # 引入嵌入以后的维度数据
num_layers = 2 # rnn的层数
batch_size = 1 # 批次
seq_len = 5 # 序列长度（五个样本数据 即 五个单词）

In [67]:
# 创建样本数据
idx2char = ['e', 'h', 'l', 'o']
x_data = [[1, 0, 2, 2, 3]] # (batch, seq_len)
y_data = [3, 1, 2, 3, 2] # (batch * seq_len)
inputs = torch.LongTensor(x_data)
labels = torch.LongTensor(y_data)


In [68]:
# 构建模型
class RnnEmbeddingModel(torch.nn.Module):
    def __init__(self):
        super(RnnEmbeddingModel,self).__init__()
        # 编写嵌入
        self.embedding = torch.nn.Embedding(input_size,embedding_size)
        # 创建rnn模型
        self.rnn_emb = torch.nn.RNN(input_size = embedding_size,hidden_size = hidden_size,num_layers= num_layers,batch_first = True)
        # 使用线性层作为最终的预测
        self.fc = torch.nn.Linear(hidden_size,num_class)

    # 前向传播过程
    def forward(self,input):
        '''
        hidden shape :  torch.Size([2, 1, 8])
        input shape :  torch.Size([1, 5, 10])
        out shape :  torch.Size([1, 5, 8])
        out shape :  torch.Size([1, 5, 4])
        '''
        # 初始化一个h0  shape: num_layers * batch_size * hidden_size
        hidden = torch.zeros(num_layers,batch_size,hidden_size)
        # print('hidden shape : ',hidden.shape)
        # print(hidden)

        # 进行embedding操作 shape: batch_size * seq_len * embedding_size
        input = self.embedding(input)
        # print('input shape : ',input.shape)
        # print(input)

        # 使用rnn进行计算 shape: batch_size * seq_len * output_size
        out,_ = self.rnn_emb(input,hidden)
        # print('out shape : ',out.shape)
        # print(out)

        # 通过线性层 shape: batch_size * seq_len * num_class
        out = self.fc(out)
        # print(out)
        # print('out shape : ',out.shape)
        
        # 调整维度
        return out.view(-1,num_class)

      

In [69]:
# 实例化模型
rnn_Embedding = RnnEmbeddingModel()

In [70]:
# 创建损失函数
criterion = torch.nn.CrossEntropyLoss()

In [71]:
# 创建优化器
optimizer = torch.optim.Adam(rnn_Embedding.parameters(), lr=0.05)


In [72]:
# 开始训练过程
epochs = 20
for epoch in range(epochs):
    # 优化器归零
    optimizer.zero_grad()
    # 使用模型进行计算
    outputs = rnn_Embedding(inputs)
    # 计算损失
    loss = criterion(outputs,labels)
    # 沿着构造的计算图进行反向传播
    loss.backward()
    # 进行优化
    optimizer.step()

    # 获取每一个序列所得的最大值
    _, idx = outputs.max(dim=1)
    idx = idx.data.numpy()
    
    print('Predicted: ', ''.join([idx2char[x] for x in idx]), end='')
    print(', Epoch [%d/%d] loss = %.3f' % (epoch + 1,epochs, loss.item()))


Predicted:  lolle, Epoch [1/20] loss = 1.401
Predicted:  ohlol, Epoch [2/20] loss = 1.069
Predicted:  ohlol, Epoch [3/20] loss = 0.834
Predicted:  ohlol, Epoch [4/20] loss = 0.589
Predicted:  ohlol, Epoch [5/20] loss = 0.392
Predicted:  ohlol, Epoch [6/20] loss = 0.250
Predicted:  ohlol, Epoch [7/20] loss = 0.158
Predicted:  ohlol, Epoch [8/20] loss = 0.102
Predicted:  ohlol, Epoch [9/20] loss = 0.067
Predicted:  ohlol, Epoch [10/20] loss = 0.045
Predicted:  ohlol, Epoch [11/20] loss = 0.031
Predicted:  ohlol, Epoch [12/20] loss = 0.022
Predicted:  ohlol, Epoch [13/20] loss = 0.016
Predicted:  ohlol, Epoch [14/20] loss = 0.012
Predicted:  ohlol, Epoch [15/20] loss = 0.009
Predicted:  ohlol, Epoch [16/20] loss = 0.007
Predicted:  ohlol, Epoch [17/20] loss = 0.005
Predicted:  ohlol, Epoch [18/20] loss = 0.004
Predicted:  ohlol, Epoch [19/20] loss = 0.004
Predicted:  ohlol, Epoch [20/20] loss = 0.003
